In [1]:
import numpy as np
import pandas as pd

In [2]:
lenses = pd.read_table("./lenses.txt", sep="\s+")
lenses = lenses.drop('id', axis = 1)
dataset = lenses.values.tolist()

# CART(分类树）

In [3]:
#dataset:原始数据集
#axis:第几个特征（就是数据集中的第几列）
#value:特征的值，例如特征age,age=1,1就是特征的值
def split_dataset(dataset, axis, value):
    new_dataset = []
    for data in dataset:
        if data[axis] == value:
            t = data[:axis]
            t.extend(data[axis+1:])
            new_dataset.append(t)
            #相当于删除第axis列，得到一个新的数据集
    return new_dataset

In [4]:
#选出出现得最多得特征
#labels_list:分类结果，即数据集最后一列
def voting(labels_list):
    features = {}
    #计算每种分类结果出现的次数
    for label in labels_list:
        if label not in features.keys():
            features[label] = 0
        features[label] = features[label] + 1
    #选出分类结果出现得最多的次数
    type_name = sorted(features.items(), reverse = True)
    #返回出现最多的分类结果
    return type_name[0][0]

In [5]:
def CART_chooseBestFeatureToSplit(dataset):
    #特征个数
    feat_len = len(dataset[0]) - 1
    #设置个很大的基尼系数
    bestGini = 999999.0
    #初始化最优特征
    bestFeature = -1
    #遍历特征得到各个特征的特征值
    for i in range(feat_len):
        featList = [example[i] for example in dataset]
        uniqueVals = set(featList)
        gini = 0.0
        #遍历特征值
        for value in uniqueVals:
            sub_dataset = split_dataset(dataset, i, value)
            prob = len(sub_dataset) / float(len(dataset))
            subp = len(split_dataset(sub_dataset, -1, value)) / float(len(sub_dataset))
        #加权求和
        gini += prob * (1.0 - pow(subp, 2) - pow(1 - subp, 2))
        if (gini < bestGini):
            bestGini = gini
            bestFeature = i
    return bestFeature

In [6]:
#labels:特征('age', 'prescription', 'astigmtic', 'rate')
def creat_tree(dataset, labels):
    labels_list = []
    #将分类结果(数据集最后一列）放置列表中
    for data in dataset:
        labels_list.append(data[-1])
    #如果只有一种分类结果
    if labels_list.count((labels_list[0])) == len(labels_list):
        return labels_list[0]
    #如果只剩下一列（即剩下数据集最后一列）,选出出现最多次的分类结果
    if len(dataset[0]) == 1:
        return voting(labels_list)
    #获得最优特征的位置
    best_feat = CART_chooseBestFeatureToSplit(dataset)
    #获得最优特征
    best_feat_label = labels[best_feat]
    tree = {best_feat_label:{}}
    #把最优特征从分类结果中删除
    del(labels[best_feat])
    #获取最优特征值
    #feat_values = []
    for data in dataset:
#         if data[best_feat] not in feat_values:
#             feat_values.append(data[best_feat])
        feat_values = [example[best_feat] for example in dataset]
        unique_vals = set(feat_values)
    #遍历最优特征值
    for feat in feat_values:
        #用来存放除去最优特征后的特征
        sub_labels = labels[:]
        #递归求解
        tree[best_feat_label][feat] = creat_tree(split_dataset(dataset, best_feat, feat), sub_labels)
    return tree

In [7]:
def classify_tree(input_tree, feat_labels, test_vec):
    first_str = list(input_tree.keys())[0]
    second_dict = input_tree[first_str]
    # 将标签字符串转换为索引
    feat_index = feat_labels.index(first_str)
    class_label = {}
    # 递归遍历整棵树
    for key in second_dict.keys():
        if test_vec[feat_index] == key:
            if type(second_dict[key]).__name__ == 'dict':
                class_label = classify_tree(second_dict[key], feat_labels, test_vec)
            else:
                class_label = second_dict[key]
    return class_label

# 调试

In [8]:
labels = [ 'age', 'prescription', 'astigmtic', 'rate']
tree = creat_tree(dataset, labels)

In [9]:
labels = [ 'age', 'prescription', 'astigmtic', 'rate']
test = lenses.drop('type', axis = 1).values.tolist()
results = []
for i in range(len(test)):
    results.append(classify_tree(tree, labels, test[i]))

In [10]:
results

[3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 3, 3, 3, 3, 1, 3, 2, 3, 3]

In [11]:
lenses['type'].values.tolist()

[3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 3, 3, 3, 3, 1, 3, 2, 3, 3]